In [1]:
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
from datetime import datetime, timedelta

# train, test, sample_submission, building 정보 로드
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
sample_submission = pd.read_csv('./sample_submission.csv')
building_df = pd.read_csv('./building_info.csv')

C:\Users\wkdal\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def classify_weekdays(start_date, end_date):
    date_format = "%Y-%m-%d"
    start = datetime.strptime(start_date, date_format)
    end = datetime.strptime(end_date, date_format)
    
    classification = []
    current_date = start
    
    while current_date <= end:
        if current_date.weekday() < 5:  # 0: 월요일, 1: 화요일, ..., 4: 금요일
            classification.append(0)  # 평일
        else:
            classification.append(1)  # 주말
        
        current_date += timedelta(days=1)
    
    return classification

In [3]:
data = {'date': pd.date_range(start='2022-06-01', end='2022-08-24')}
train = pd.DataFrame(data)
t_data = {'date': pd.date_range(start='2022-08-25', end='2022-08-31')}
test = pd.DataFrame(t_data)

In [4]:
start_date = "2022-06-01"
end_date = "2022-08-24"
test_start_date = "2022-08-25"
test_end_date = "2022-08-31"
train['holidays'] = classify_weekdays(start_date, end_date)
train_df['holidays'] = train['holidays']
test['holidays'] = classify_weekdays(test_start_date, test_end_date)
test_df['holidays'] = test['holidays']

In [5]:
train_df['일시'] = pd.to_datetime(train_df['일시'], format='%Y%m%d %H')
test_df['일시'] = pd.to_datetime(test_df['일시'], format='%Y%m%d %H')

train_df['hour'] = train_df['일시'].dt.hour
train_df['day'] = train_df['일시'].dt.day
train_df['month'] = train_df['일시'].dt.month
train_df['year'] = train_df['일시'].dt.year
train_df['sin_time'] = np.sin(2*np.pi*train_df.hour/24)
train_df['cos_time'] = np.cos(2*np.pi*train_df.hour/24)

test_df['hour'] = test_df['일시'].dt.hour
test_df['day'] = test_df['일시'].dt.day
test_df['month'] = test_df['일시'].dt.month
test_df['year'] = test_df['일시'].dt.year
test_df['sin_time'] = np.sin(2*np.pi*test_df.hour/24)
test_df['cos_time'] = np.cos(2*np.pi*test_df.hour/24)
train_df['THI'] = 9/5*train_df['기온(C)'] - 0.55*(1-train_df['습도(%)']/100)*(9/5*train_df['습도(%)']-26)+32
test_df['THI'] = 9/5*test_df['기온(C)'] - 0.55*(1-test_df['습도(%)']/100)*(9/5*test_df['습도(%)']-26)+32

In [6]:
ensemble_predictor = TabularPredictor(label=label, problem_type='regression')
for fold_idx, fold_data in enumerate(train_folds):
    print(f"Training fold {fold_idx + 1}")
    
    data = {'일시': pd.date_range(start=date_ranges[fold_idx][0], end=date_ranges[fold_idx][1])}
    fold_train = pd.DataFrame(data)
    
    holidays = classify_weekdays(date_ranges[fold_idx][0], date_ranges[fold_idx][1])
    fold_train['holidays'] = holidays
    
    # '기온(C)' 열 생성 또는 가져오기 (실제 데이터에 맞게)
    fold_train['기온(C)'] = np.random.randint(10, 30, size=len(fold_train))
    
    fold_train['hour'] = fold_train['일시'].dt.hour
    fold_train['day'] = fold_train['일시'].dt.day
    fold_train['month'] = fold_train['일시'].dt.month
    fold_train['year'] = fold_train['일시'].dt.year
    fold_train['sin_time'] = np.sin(2*np.pi*fold_train.hour/24)
    fold_train['cos_time'] = np.cos(2*np.pi*fold_train.hour/24)
    fold_train['THI'] = 9/5*fold_train['기온(C)'] - 0.55*(1-fold_train['습도(%)']/100)*(9/5*fold_train['습도(%)']-26)+32
    fold_train = fold_train.drop(columns=['일조(hr)', '일사(MJ/m2)', '강수량(mm)', '풍속(m/s)'])
    
    fold_train_data = TabularDataset(fold_train)
    ensemble_predictor.fit(fold_train_data)

NameError: name 'label' is not defined

In [ ]:
test_df['일시'] = pd.to_datetime(test_df['일시'], format='%Y%m%d %H')
test_df['hour'] = test_df['일시'].dt.hour
test_df['day'] = test_df['일시'].dt.day
test_df['month'] = test_df['일시'].dt.month
test_df['year'] = test_df['일시'].dt.year
test_df['sin_time'] = np.sin(2*np.pi*test_df.hour/24)
test_df['cos_time'] = np.cos(2*np.pi*test_df.hour/24)
test_df['THI'] = 9/5*test_df['기온(C)'] - 0.55*(1-test_df['습도(%)']/100)*(9/5*test_df['습도(%)']-26)+32
test_data = TabularDataset(test_df)

predictions = ensemble_predictor.predict(test_data)

sample_submission['answer'] = predictions['전력소비량(kWh)']
sample_submission.to_csv('./baseline_submission_kfold.csv', index=False)